<a href="https://colab.research.google.com/github/saksham-v25/ML/blob/main/K-means/big_basket_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from difflib import get_close_matches


In [ ]:
# Load your full dataset
df = pd.read_csv("/content/BigBasket Products.csv")  # Replace with actual file path or upload

# Sample only 5000 rows to avoid memory crash
df = df.sample(n=5000, random_state=42).reset_index(drop=True)


In [ ]:
# Drop rows where product name is missing (NaN)
df = df[df['product'].notnull()]

# Convert all product names to string
df['product'] = df['product'].astype(str)


<ipython-input-3-107b286bbab7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['product'] = df['product'].astype(str)


In [1]:
# Combine relevant text fields
df['features'] = (df['product'].astype(str) + " " +
                  df['category'].astype(str) + " " +
                  df['description'].astype(str)).str.lower()


NameError: name 'df' is not defined

In [ ]:
tfidf = TfidfVectorizer(stop_words='english', max_df=0.8, min_df=5, max_features=10000)
tfidf_matrix = tfidf.fit_transform(df['features'])


In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)



In [ ]:
def recommend_products(product_name, num_recommendations=5):
    product_list = df['product'].str.lower().tolist()
    close_matches = get_close_matches(product_name.lower(), product_list, n=1, cutoff=0.6)

    if not close_matches:
        return ["No similar products found."]

    matched_product = close_matches[0]
    product_idx = df[df['product'].str.lower() == matched_product].index[0]

    similarity_scores = list(enumerate(cosine_sim[product_idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    recommendations = []
    for i in similarity_scores[1:num_recommendations + 1]:
        recommended_product = df.iloc[i[0]]['product']
        recommendations.append(recommended_product)

    return recommendations


In [ ]:
product_name = "Mango"  # Try variations like "soap", "tea", "chips"
recommendations = recommend_products(product_name)

print(f"\nTop recommendations for '{product_name}':")
for rec in recommendations:
    print("-", rec)



Top recommendations for 'Mango':
- Tru Orange
- Fruit Power Juice - Mango/Aam
- FrutORS - Orange
- Soft Drink - Lime Flavoured 2.25l+Mango drink 600ml
- Fruit Power - Litchi


PHASE 2

In [ ]:
# Downgrade NumPy to avoid crashes with Surprise
!pip install numpy==1.24.3
!pip install scikit-surprise


  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505209 sha256=9034ca4013aa293c5681c481c250702296cbe53a3918059fa68b216004c255e0
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
import pandas as pd


In [ ]:
!pip uninstall -y numpy scikit-surprise


Found existing installation: numpy 1.24.3
Uninstalling numpy-1.24.3:
  Successfully uninstalled numpy-1.24.3
Found existing installation: scikit-surprise 1.1.4
Uninstalling scikit-surprise-1.1.4:
  Successfully uninstalled scikit-surprise-1.1.4


In [ ]:
#!pip install numpy==1.24.3 scikit-surprise --no-cache-dir --force-reinstall


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 124.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 154.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 185.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 257.5 MB/s eta 0:00:00
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505217 sha256=079a5c774969d37cbb585ba071e59a8a6c386a96593734d5182fbc437067f67f
  Stored in directory: /tmp/pip-ephem-wheel-cache-yrilbx1w/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise
  Attempting uninstall: joblib
    Found existing installation: joblib 1.4.2
    Uninstalling joblib-1.4.2:
      Su

In [ ]:
# Re-import necessary libraries to make sure everything is working
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
import pandas as pd

print("🔥 All set. Surprise and NumPy are working!")


🔥 All set. Surprise and NumPy are working!


In [ ]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
import pandas as pd


In [ ]:
import random

# Create 1000 fake user IDs
user_ids = [f"user_{i}" for i in range(1, 1001)]

# Create a list to store user-product-rating interactions
ratings_data = []

# Each product gets rated by 1 to 5 users randomly
for index, row in df.iterrows():
    for _ in range(random.randint(1, 5)):
        user = random.choice(user_ids)
        rating = row['rating']
        if pd.notnull(rating):  # Only include if rating exists
            ratings_data.append([user, row['product'], float(rating)])

# Convert to a new DataFrame
ratings_df = pd.DataFrame(ratings_data, columns=["user_id", "product", "rating"])

# Check it
ratings_df.head()


,user_id,product,rating
0,user_958,"Knutzella Bar Bites - Mini, Gluten-free, Vegan",2.0
1,user_895,"Knutzella Bar Bites - Mini, Gluten-free, Vegan",2.0
2,user_87,"Knutzella Bar Bites - Mini, Gluten-free, Vegan",2.0
3,user_831,"Knutzella Bar Bites - Mini, Gluten-free, Vegan",2.0
4,user_618,Standard Large - 34 Diaper Pants,4.2


In [ ]:
from surprise import Dataset, Reader

# Define rating scale
reader = Reader(rating_scale=(0, 5))

# Load ratings into Surprise
data = Dataset.load_from_df(ratings_df[['user_id', 'product', 'rating']], reader)


In [ ]:
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Train-test split
trainset, testset = train_test_split(data, test_size=0.2)

# Model
model = SVD()
model.fit(trainset)

# Predict
predictions = model.test(testset)
accuracy.rmse(predictions)


RMSE: 0.6361


0.636058493405894

In [ ]:
df.columns


Index(['index', 'product', 'category', 'sub_category', 'brand', 'sale_price',
       'market_price', 'type', 'rating', 'description', 'features'],
      dtype='object')

In [ ]:
# Create 1000 dummy users
user_ids = [f"user_{i}" for i in range(1, 1001)]

# Randomly assign users to products
ratings_data = []
for index, row in df.iterrows():
    for _ in range(random.randint(1, 5)):  # Each product gets rated by 1–5 users
        user = random.choice(user_ids)
        rating = row['rating']
        if pd.notnull(rating):
            ratings_data.append([user, row['product'], float(rating)])

# Convert to DataFrame
ratings_df = pd.DataFrame(ratings_data, columns=["user_id", "product", "rating"])

print(ratings_df.head())


    user_id                                         product  rating
0  user_879  Knutzella Bar Bites - Mini, Gluten-free, Vegan     2.0
1  user_477  Knutzella Bar Bites - Mini, Gluten-free, Vegan     2.0
2  user_261  Knutzella Bar Bites - Mini, Gluten-free, Vegan     2.0
3  user_464  Knutzella Bar Bites - Mini, Gluten-free, Vegan     2.0
4  user_842                Standard Large - 34 Diaper Pants     4.2


In [ ]:
# Recommend top N products to a user
def recommend_for_user(user_id, df, model, n=5):
    all_products = df['product'].unique()
    already_rated = ratings_df[ratings_df['user_id'] == user_id]['product'].tolist()
    products_to_predict = [p for p in all_products if p not in already_rated]

    predicted_ratings = []
    for product in products_to_predict:
        pred = model.predict(user_id, product)
        predicted_ratings.append((product, pred.est))

    # Sort by predicted rating
    recommended = sorted(predicted_ratings, key=lambda x: x[1], reverse=True)[:n]
    return [prod for prod, _ in recommended]

# Test it
test_user = ratings_df['user_id'].iloc[0]
print(f"Recommended products for {test_user}:")
print(recommend_for_user(test_user, df, model))


Recommended products for user_879:
['Utensils Drying Microfiber Mat - Blue, BB1430', 'Vetiver Hydrosol Floral Water', 'Whisky Juice Glass Set - Sirocco Rock', 'Blush - Shine & Glow Serum', 'Grooming Comb - GRC1004, Colour May Vary']


PHASE 3

In [ ]:
def hybrid_recommendation(user_id, product_name, tfidf_weight=0.5, svd_weight=0.5, n=5):
    # TF-IDF recommendations
    tfidf_recs = recommend_products(product_name, num_recommendations=20)  # content-based
    tfidf_scores = {prod: (20 - i) * tfidf_weight for i, prod in enumerate(tfidf_recs)}

    # SVD predictions
    svd_preds = []
    for prod in tfidf_recs:
        try:
            pred = model.predict(user_id, prod).est
            svd_preds.append((prod, pred * svd_weight))
        except:
            continue

    # Combine scores
    final_scores = {}
    for prod, svd_score in svd_preds:
        final_scores[prod] = tfidf_scores.get(prod, 0) + svd_score

    # Sort and recommend
    sorted_recs = sorted(final_scores.items(), key=lambda x: x[1], reverse=True)[:n]
    return [prod for prod, _ in sorted_recs]

# Test hybrid system
test_user = ratings_df['user_id'].iloc[5]
print(f"Hybrid recommendations for {test_user}:")
print(hybrid_recommendation(test_user, "tea"))


Hybrid recommendations for user_258:
['Paper - A4, Excel Bond, 85 GSM', 'ScalGo Descaling Powder', 'Oil Pastels - 25+2 Colour Set, With Scraper, Diameter', 'Food Colour - Yellow', 'Laundry Powder']
